<a href="https://www.kaggle.com/code/mayurdevarajpatil/bikedekho-web-scrapping?scriptVersionId=138448131" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import re

In [2]:
h={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
bike = requests.get('https://www.bikedekho.com/new-bikes#all_brands',headers=h)
bike

<Response [200]>

In [3]:
h={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
data = requests.get('https://www.bikedekho.com/new-bikes#all_brands',headers=h)
data1=bs(data.text)
data2=data1.find_all('a',class_='BrIconNewCar')
links=[]
for i in data2:
    links.append(i['href'].replace('/',''))
links

['tvs-bikes',
 'bajaj-bikes',
 'royal-enfield-bikes',
 'yamaha-bikes',
 'honda-bikes',
 'hero-bikes',
 'suzuki-bikes',
 'ktm-bikes',
 'jawa-motorcycles-bikes',
 'mahindra-bikes',
 'harley-davidson-bikes',
 'ducati-bikes',
 'kawasaki-bikes',
 'benelli-bikes',
 'triumph-bikes',
 'indian-bikes',
 'bmw-bikes',
 'aprilia-bikes',
 'revolt-bikes',
 'ultraviolette-bikes',
 'pure-ev-bikes',
 'komaki-bikes',
 'joy-e-bikes',
 'moto-guzzi-bikes',
 'keeway-bikes',
 'srivaru-motors-bikes',
 'tork-bikes',
 'rowwet-bikes',
 'tunwal-bikes',
 'yezdi-motorcycles-bikes',
 'qj-motor-bikes',
 'power-ev-bikes',
 'adms-bikes',
 'one-electric-motorcycles-bikes',
 'odysse-electric-bikes',
 'oben-bikes',
 'moto-morini-bikes',
 'matter-ev-bikes',
 'aeroride-bikes',
 'kabira-mobility-bikes',
 'husqvarna-bikes',
 'hop-electric-bikes',
 'evtric-motors-bikes',
 'earth-energy-ev-bikes',
 'cyborg-bikes',
 'cfmoto-bikes',
 'birla-e-bike-bikes',
 'atumobile-bikes',
 'zontes-bikes']

In [4]:
links.remove('komaki-bikes')
links.remove('tunwal-bikes')
links.remove('aeroride-bikes')
links.remove('evtric-motors-bikes')
links.remove('earth-energy-ev-bikes')
links.remove('birla-e-bike-bikes')
links.remove('atumobile-bikes')
links.remove('zontes-bikes')
links.remove('honda-bikes')

In [5]:
columns=['Bike_Name','Price','Milage (in KMPL)','Engine (in cc)','Fuel Type']
df=pd.DataFrame(columns=columns)
for link in links:
    Brands = requests.get(f'https://www.bikedekho.com/{link}',headers=h)
    bikes1=bs(Brands.text)
    
    bikes2=bikes1.find_all('ul',class_='bikelist')[0]

    
    Company=[]
    l=link.replace('-bikes','')
    l1=l.replace('-motorcycles','').upper()
    Company.append(l1)
  
    
    bike=bikes2.find_all('h3')
    Bike_names=[]
    for i in bike:
        Bike_names.append(i.text)

        
    prices=bikes2.find_all('div',class_='price')
    price_1=[]
    price=[]
    for j in prices:
        price0=((str(j.text).replace('*','')))
        if '-' in price0:
            price_1.append(price0[(price0.index('-'))+2:])
        else:
            price0=((price0.replace('Rs.','')))
            price_1.append(price0)
    for i in price_1:
        if 'Lakh' in i:
            price.append(float(i.split(' ')[0])*100000)
        else:
            i1=i.replace(',','')
            price.append(float(i1))

        
    
    milages=bikes2.find_all('div',class_='dotlist')
    milage=[]
    Type=[]
    for i in milages:
        temp=(i.find('span')).text
        if 'kmpl' not in temp and 'Kmpl' not in temp:
            milage.append('-')
            Type.append('Electric')
        else:
            milage.append(float(temp.strip().split()[0]))
            Type.append('Petrol')

    
    CCs=bikes2.find_all('div',class_='dotlist')
    CC=[]
    for i in CCs:
        temp=i.find_all('span')[1].text
        if 'cc' in temp:
            CC.append(float(temp.strip().split()[0])) 
        else:
            CC.append('-')  
    
    for i in Bike_names:
        DF=pd.DataFrame({'Bike_Name':Bike_names,'Price':price,'Milage (in KMPL)':milage,'Engine (in cc)':CC,'Fuel Type':Type})
    df=pd.concat([df,DF])
    
df=df.reset_index(drop=True)
df['Company']=df['Bike_Name'].map(lambda x: x.split()[0])
df=df[['Company','Bike_Name','Price','Milage (in KMPL)','Engine (in cc)','Fuel Type']]
df

,Company,Bike_Name,Price,Milage (in KMPL),Engine (in cc),Fuel Type
0,TVS,TVS Raider,100000.0,67.0,124.8,Petrol
1,TVS,TVS NTORQ 125,105000.0,54.33,124.8,Petrol
2,TVS,TVS Apache RTR 160,126000.0,47.0,159.7,Petrol
3,TVS,TVS Jupiter,89648.0,50.0,109.7,Petrol
4,TVS,TVS iQube Electric,161000.0,-,-,Electric
...,...,...,...,...,...,...
304,Cyborg,Cyborg Bob-e,94999.0,-,-,Electric
305,CFMoto,CFMoto 650GT,559000.0,20.0,649.3,Petrol
306,CFMoto,CFMoto 300NK,229000.0,33.0,292.4,Petrol
307,CFMoto,CFMoto 650NK,429000.0,20.0,649.3,Petrol
